In [ ]:
# Class that implements the variations of ResNet presented in the original paper
class Resnet(object):
    def __init__(self, opt):
        self.use_batchnorm = opt.use_batchnorm
        self.block_filters = opt.block_filters
        self.block_strides = opt.block_strides
        self.block_sizes = opt.block_sizes
        self.block = basicblock if opt.block_type == 'basicblock' else bottleneck
        self.opt = opt
        if opt.preset == '50':
            self.block = bottleneck
            self.block_filters = [ 256, 512, 1024, 2048 ]
            self.block_sizes = [3, 4, 6, 3]
            self.block_strides = [ 1, 2, 2, 2 ]
        elif opt.preset == '18':
            self.block = bottleneck
            self.block_filters = [ 64, 128, 256, 512 ]
            self.block_sizes = [2, 2, 2, 2]
            self.block_strides = [ 1, 2, 2, 2 ]
        
    def build_net(self, images, labels, is_training):
        with tf.variable_scope('block0') as scope:
            x = conv(images, 7, 64, 2, name='conv1', use_bias = not self.use_batchnorm)
            if self.use_batchnorm:
                x = bn(x, is_training)
            x = relu(x)
            x = maxpool(x, 3, 2)

        blockno = 1
        for size, filters, stride in zip(self.block_sizes, self.block_filters,
                                         self.block_strides):
            print('Making basic block {}'.format(blockno))
            with tf.variable_scope('block{}'.format(blockno)) as scope:
                for i in range(size):
                    x = self.block(x, filters, stride if i == 0 else 1,
                                   is_training, name='block{}'.format(i+1),
                                   use_batchnorm = self.use_batchnorm,
                                   use_bias = not self.use_batchnorm)
                blockno = blockno + 1

        with tf.variable_scope('output'):
            x = tf.reduce_mean(x, axis = [1,2])
            x = fc(x, self.opt.num_classes)

        preds = tf.nn.softmax(x)

        wd_loss = tf.add_n([ tf.nn.l2_loss(v) for v in tf.trainable_variables()
                             if 'kernel'])*0.0001
        loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=x))
        return preds, loss + wd_loss


In [ ]:
class MiResNet(Resnet):
    def build_subnet(self, images, is_training):
        with tf.variable_scope('block0') as scope:
            x = conv(images, 7, 64, 2, name='conv1', use_bias = not self.use_batchnorm)
            if self.use_batchnorm:
                x = bn(x, is_training)
            x = relu(x)
            x = maxpool(x, 3, 2)

        blockno = 1
        for size, filters, stride in zip(self.block_sizes, self.block_filters,
                                         self.block_strides):
            print('Making basic block {}'.format(blockno))
            with tf.variable_scope('block{}'.format(blockno)) as scope:
                for i in range(size):
                    x = self.block(x, filters, stride if i == 0 else 1,
                                   is_training, name='block{}'.format(i+1),
                                   use_batchnorm = self.use_batchnorm,
                                   use_bias = not self.use_batchnorm)
                blockno = blockno + 1

        with tf.variable_scope('output'):
            x = tf.reduce_mean(x, axis = [1,2])
        return x
    def build_net(self, inputs, labels, is_training):
        with tf.variable_scope('shared'):
            north_output = self.build_subnet(inputs[0], is_training)
        with tf.variable_scope('shared', reuse = True):
            west_output = self.build_subnet(inputs[1], is_training)
        with tf.variable_scope('shared', reuse = True):
            south_output = self.build_subnet(inputs[2], is_training)
        with tf.variable_scope('shared', reuse = True):
            east_output = self.build_subnet(inputs[3], is_training)

        with tf.variable_scope('combine'):
            x = tf.concat((north_output, west_output, south_output, east_output),
                          axis = 1)
            x = fc(x, self.opt.num_classes)
        
        preds = tf.nn.softmax(x)
        
        wd_loss = tf.add_n([ tf.nn.l2_loss(v) for v in tf.trainable_variables()
                             if 'kernel'])*0.0001
        loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=x))
        return preds, loss + wd_loss